# SwellSight Enhanced: FLUX ControlNet Synthetic Generation

**Project:** SwellSight Wave Analysis Model - Enhanced Pipeline
**Pipeline Stage:** FLUX-based Real-to-Synthetic Image Generation

## Overview
This enhanced notebook implements the next-generation synthetic image generation using FLUX.1-dev with ControlNet-Depth integration. It replaces the previous Stable Diffusion pipeline with state-of-the-art FLUX models for superior image quality and depth conditioning.

## Key Improvements
1. **FLUX.1-dev Integration:** Advanced diffusion model for high-quality image generation
2. **FLUX ControlNet-Depth:** Precise depth-conditioned generation
3. **Mixed Precision Training:** Improved performance and memory efficiency
4. **Dynamic Memory Management:** Adaptive batch sizing and GPU optimization
5. **Quality Validation:** Comprehensive synthetic vs real data comparison

## Prerequisites
* **Enhanced Setup Completed:** Updated dependencies and configuration from notebook 01
* **Depth Maps Available:** Depth-Anything-V2 results from notebook 03
* **Augmentation Parameters:** Enhanced parameters from notebook 04
* **GPU Runtime:** Requires high-memory GPU (A100 recommended for FLUX.1-dev)

In [ ]:
# @title 1. Environment Setup & Enhanced Dependencies
# Enhanced installation for FLUX.1-dev and advanced features

import sys
import os
import subprocess
import torch
from pathlib import Path

# Add utils to path for shared functionality
sys.path.append('./utils')

def install_enhanced_packages():
    """Install enhanced packages for FLUX.1-dev support"""
    packages = [
        "diffusers>=0.30.0",  # Latest for FLUX support
        "transformers>=4.44.0",  # Updated for FLUX tokenizers
        "accelerate>=0.33.0",  # Enhanced acceleration
        "torch>=2.4.0",  # Latest PyTorch
        "torchvision>=0.19.0",
        "opencv-python",
        "pillow>=10.0.0",
        "numpy>=1.24.0",
        "scipy>=1.11.0",
        "scikit-image>=0.21.0",
        "matplotlib>=3.7.0",
        "tqdm>=4.65.0",
        "psutil>=5.9.0",  # Memory monitoring
        "sentencepiece>=0.1.99",  # For FLUX tokenization
    ]
    
    # Try to install xformers for memory optimization
    xformers_packages = [
        "xformers>=0.0.22",
        "flash-attn>=2.3.0",  # Flash attention for FLUX
    ]
    
    print("📦 Installing enhanced dependencies for FLUX.1-dev...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages)
    
    # Try to install optional optimization packages
    for pkg in xformers_packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
            print(f"✅ Installed {pkg}")
        except subprocess.CalledProcessError:
            print(f"⚠️ Could not install {pkg} - continuing without")
    
    print("✅ Enhanced dependencies installed.")

# Check and install dependencies
try:
    import diffusers
    from diffusers import FluxPipeline, FluxControlNetPipeline
    print(f"✅ Diffusers version: {diffusers.__version__}")
except ImportError:
    install_enhanced_packages()
    import diffusers
    from diffusers import FluxPipeline, FluxControlNetPipeline

# Enhanced GPU detection and memory info
if not torch.cuda.is_available():
    print("❌ ERROR: No GPU detected. FLUX.1-dev requires GPU acceleration.")
    print("   Please ensure you have a CUDA-compatible GPU and drivers installed.")
    raise RuntimeError("GPU required for FLUX.1-dev")
else:
    device_name = torch.cuda.get_device_name(0)
    memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"✅ GPU Detected: {device_name}")
    print(f"📊 GPU Memory: {memory_gb:.1f} GB")
    
    if memory_gb < 16:
        print("⚠️ WARNING: FLUX.1-dev works best with 16GB+ GPU memory.")
        print("   Consider using smaller batch sizes or CPU offloading.")

In [ ]:
# @title 2. Load Configuration & Shared Utilities
import json
import logging
from pathlib import Path
import numpy as np
from PIL import Image
import cv2
from dataclasses import dataclass
from typing import Dict, Any, Optional, List, Tuple

# Import shared utilities
try:
    from config_manager import ConfigManager
    from memory_optimizer import MemoryOptimizer
    from data_validator import DataValidator
    from error_handler import ErrorHandler
    from progress_tracker import ProgressTracker
    from data_flow_manager import DataFlowManager
    print("✅ Shared utilities imported successfully")
except ImportError as e:
    print(f"⚠️ Could not import shared utilities: {e}")
    print("   Continuing with basic functionality...")
    # Define minimal fallback classes
    class ConfigManager:
        @staticmethod
        def load_config(path='config.json'):
            with open(path, 'r') as f:
                return json.load(f)
    
    class MemoryOptimizer:
        @staticmethod
        def get_optimal_batch_size(available_memory, item_size_mb=100):
            return max(1, int(available_memory * 0.8 / item_size_mb))
        
        @staticmethod
        def cleanup_memory():
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
    
    class DataValidator:
        @staticmethod
        def validate_image_quality(image_path):
            return {'score': 0.8, 'valid': True}
    
    class ErrorHandler:
        @staticmethod
        def handle_gpu_memory_error(operation, fallback_func=None):
            print(f"⚠️ GPU memory error in {operation}")
            if fallback_func:
                return fallback_func()
            return None
    
    class ProgressTracker:
        def __init__(self, total, description="Processing"):
            from tqdm import tqdm
            self.pbar = tqdm(total=total, desc=description)
        
        def update(self, n=1):
            self.pbar.update(n)
        
        def close(self):
            self.pbar.close()
    
    class DataFlowManager:
        @staticmethod
        def load_previous_results(stage_name, required_files=None):
            # Basic implementation
            return {}
        
        @staticmethod
        def save_stage_results(data, stage_name, metadata=None):
            # Basic implementation
            pass

# Load configuration
try:
    config = ConfigManager.load_config('config.json')
    print("✅ Configuration loaded successfully")
    print(f"📋 Pipeline: {config['pipeline']['name']} v{config['pipeline']['version']}")
except Exception as e:
    print(f"❌ Configuration Error: {e}")
    print("   Using default configuration...")
    config = {
        'models': {
            'base_model': {'name': 'black-forest-labs/FLUX.1-dev'},
            'controlnet_model': {'name': 'Shakker-Labs/FLUX.1-dev-ControlNet-Depth'},
            'mixed_precision': True
        },
        'processing': {'batch_size': 'auto'},
        'paths': {'output_dir': './outputs'}
    }

# Setup paths
OUTPUT_DIR = Path(config['paths']['output_dir'])
SYNTHETIC_DIR = OUTPUT_DIR / 'synthetic'
IMAGES_DIR = SYNTHETIC_DIR / 'images'
METADATA_DIR = SYNTHETIC_DIR / 'metadata'

# Create directories
for dir_path in [SYNTHETIC_DIR, IMAGES_DIR, METADATA_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"📂 Output Directory: {IMAGES_DIR}")
print(f"📊 Metadata Directory: {METADATA_DIR}")

In [ ]:
# @title 3. Enhanced FLUX ControlNet Generator Class
from diffusers import FluxControlNetPipeline, FluxControlNetModel
from diffusers.utils import load_image
import torch.nn.functional as F
from scipy import stats
import psutil

@dataclass
class EnhancedGenerationResult:
    """Enhanced result structure with comprehensive metadata"""
    synthetic_image: Image.Image
    depth_map: np.ndarray
    generation_params: Dict[str, Any]
    quality_metrics: Dict[str, float]
    processing_time: float
    memory_usage: Dict[str, float]
    model_info: Dict[str, str]

class FLUXControlNetGenerator:
    """Enhanced FLUX.1-dev ControlNet generator with advanced features"""
    
    def __init__(self, config: Dict[str, Any], device='cuda'):
        self.config = config
        self.device = device
        self.model_config = config['models']
        
        # Model identifiers
        self.base_model_id = self.model_config['base_model']['name']
        self.controlnet_id = self.model_config['controlnet_model']['name']
        
        # Pipeline components
        self.pipe = None
        self.controlnet = None
        
        # Memory and performance tracking
        self.memory_optimizer = MemoryOptimizer()
        self.error_handler = ErrorHandler()
        
        # Initialize the pipeline
        self._initialize_pipeline()
    
    def _initialize_pipeline(self):
        """Initialize FLUX ControlNet pipeline with optimizations"""
        print(f"🔄 Loading FLUX ControlNet: {self.controlnet_id}...")
        
        try:
            # Load ControlNet model
            self.controlnet = FluxControlNetModel.from_pretrained(
                self.controlnet_id,
                torch_dtype=torch.float16 if self.model_config.get('mixed_precision', True) else torch.float32
            )
            print("✅ FLUX ControlNet loaded")
            
            # Load main pipeline
            print(f"🔄 Loading FLUX.1-dev pipeline: {self.base_model_id}...")
            self.pipe = FluxControlNetPipeline.from_pretrained(
                self.base_model_id,
                controlnet=self.controlnet,
                torch_dtype=torch.float16 if self.model_config.get('mixed_precision', True) else torch.float32
            )
            
            # Apply optimizations
            self._apply_optimizations()
            
            # Move to device
            self.pipe.to(self.device)
            print("✅ FLUX pipeline initialized successfully")
            
            # Print memory usage
            if torch.cuda.is_available():
                memory_used = torch.cuda.memory_allocated() / (1024**3)
                print(f"📊 GPU Memory Used: {memory_used:.2f} GB")
        
        except Exception as e:
            print(f"❌ Pipeline initialization failed: {e}")
            raise
    
    def _apply_optimizations(self):
        """Apply various optimizations to the pipeline"""
        optimizations_applied = []
        
        # Enable memory efficient attention if available
        try:
            self.pipe.enable_xformers_memory_efficient_attention()
            optimizations_applied.append("xformers")
        except Exception:
            try:
                self.pipe.enable_attention_slicing()
                optimizations_applied.append("attention_slicing")
            except Exception:
                pass
        
        # Enable model CPU offload if configured
        if self.model_config.get('cpu_offload', False):
            try:
                self.pipe.enable_model_cpu_offload()
                optimizations_applied.append("cpu_offload")
            except Exception:
                pass
        
        # Enable sequential CPU offload for extreme memory saving
        if self.model_config.get('sequential_offload', False):
            try:
                self.pipe.enable_sequential_cpu_offload()
                optimizations_applied.append("sequential_offload")
            except Exception:
                pass
        
        if optimizations_applied:
            print(f"🚀 Optimizations enabled: {', '.join(optimizations_applied)}")
        else:
            print("⚠️ No optimizations could be applied")
    
    def preprocess_depth_map(self, depth_map: np.ndarray, target_size: Tuple[int, int] = (1024, 1024)) -> Image.Image:
        """Enhanced depth map preprocessing for FLUX ControlNet"""
        # Ensure depth map is 2D
        if len(depth_map.shape) == 3:
            depth_map = depth_map.squeeze()
        
        # Normalize to 0-1 range
        if depth_map.max() != depth_map.min():
            depth_normalized = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())
        else:
            depth_normalized = np.zeros_like(depth_map)
        
        # Apply gentle smoothing to reduce noise
        depth_smoothed = cv2.GaussianBlur(depth_normalized, (3, 3), 0.5)
        
        # Convert to uint8
        depth_uint8 = (depth_smoothed * 255).astype(np.uint8)
        
        # Resize to target size
        depth_resized = cv2.resize(depth_uint8, target_size, interpolation=cv2.INTER_LINEAR)
        
        # Convert to RGB PIL Image (FLUX expects RGB)
        depth_rgb = cv2.cvtColor(depth_resized, cv2.COLOR_GRAY2RGB)
        return Image.fromarray(depth_rgb)
    
    def calculate_quality_metrics(self, synthetic_image: Image.Image, depth_map: np.ndarray) -> Dict[str, float]:
        """Calculate comprehensive quality metrics for generated images"""
        # Convert to numpy for analysis
        img_array = np.array(synthetic_image)
        
        # Basic image quality metrics
        brightness = np.mean(img_array)
        contrast = np.std(img_array)
        
        # Color distribution metrics
        color_variance = np.var(img_array, axis=(0, 1)).mean()
        
        # Edge preservation (compare with depth edges)
        gray_img = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        img_edges = cv2.Canny(gray_img, 50, 150)
        depth_edges = cv2.Canny((depth_map * 255).astype(np.uint8), 50, 150)
        
        # Edge correlation (how well synthetic edges match depth edges)
        edge_correlation = np.corrcoef(img_edges.flatten(), depth_edges.flatten())[0, 1]
        edge_correlation = 0.0 if np.isnan(edge_correlation) else edge_correlation
        
        return {
            'brightness': float(brightness / 255.0),
            'contrast': float(contrast / 255.0),
            'color_variance': float(color_variance / 255.0),
            'edge_correlation': float(edge_correlation),
            'overall_quality': float((brightness/255 + contrast/255 + abs(edge_correlation)) / 3)
        }
    
    def generate_with_memory_management(self, depth_map: np.ndarray, params: Dict[str, Any]) -> EnhancedGenerationResult:
        """Generate synthetic image with comprehensive memory management"""
        import time
        start_time = time.time()
        
        # Record initial memory state
        initial_memory = {
            'gpu_allocated': torch.cuda.memory_allocated() / (1024**3) if torch.cuda.is_available() else 0,
            'gpu_reserved': torch.cuda.memory_reserved() / (1024**3) if torch.cuda.is_available() else 0,
            'system_memory': psutil.virtual_memory().percent
        }
        
        try:
            # Preprocess depth map
            target_size = (params.get('width', 1024), params.get('height', 1024))
            depth_image = self.preprocess_depth_map(depth_map, target_size)
            
            # Setup generation parameters
            generation_kwargs = {
                'prompt': params['prompt'],
                'control_image': depth_image,
                'height': params.get('height', 1024),
                'width': params.get('width', 1024),
                'num_inference_steps': params.get('num_inference_steps', 28),
                'guidance_scale': params.get('guidance_scale', 3.5),
                'controlnet_conditioning_scale': params.get('controlnet_conditioning_scale', 0.6),
            }
            
            # Add seed if provided
            if 'seed' in params:
                generator = torch.Generator(device=self.device).manual_seed(int(params['seed']))
                generation_kwargs['generator'] = generator
            
            # Generate image
            with torch.cuda.amp.autocast(enabled=self.model_config.get('mixed_precision', True)):
                result = self.pipe(**generation_kwargs)
            
            synthetic_image = result.images[0]
            
            # Calculate quality metrics
            quality_metrics = self.calculate_quality_metrics(synthetic_image, depth_map)
            
            # Record final memory state
            final_memory = {
                'gpu_allocated': torch.cuda.memory_allocated() / (1024**3) if torch.cuda.is_available() else 0,
                'gpu_reserved': torch.cuda.memory_reserved() / (1024**3) if torch.cuda.is_available() else 0,
                'system_memory': psutil.virtual_memory().percent
            }
            
            processing_time = time.time() - start_time
            
            return EnhancedGenerationResult(
                synthetic_image=synthetic_image,
                depth_map=depth_map,
                generation_params=params,
                quality_metrics=quality_metrics,
                processing_time=processing_time,
                memory_usage={
                    'initial': initial_memory,
                    'final': final_memory,
                    'peak_gpu_gb': final_memory['gpu_allocated']
                },
                model_info={
                    'base_model': self.base_model_id,
                    'controlnet': self.controlnet_id,
                    'device': str(self.device),
                    'mixed_precision': str(self.model_config.get('mixed_precision', True))
                }
            )
        
        except torch.cuda.OutOfMemoryError as e:
            print(f"⚠️ GPU Memory Error: {e}")
            # Attempt memory cleanup and retry with smaller parameters
            self.memory_optimizer.cleanup_memory()
            
            # Reduce parameters for retry
            params_reduced = params.copy()
            params_reduced['height'] = min(params_reduced.get('height', 1024), 768)
            params_reduced['width'] = min(params_reduced.get('width', 1024), 768)
            params_reduced['num_inference_steps'] = min(params_reduced.get('num_inference_steps', 28), 20)
            
            print("🔄 Retrying with reduced parameters...")
            return self.generate_with_memory_management(depth_map, params_reduced)
        
        except Exception as e:
            print(f"❌ Generation failed: {e}")
            raise
    
    def cleanup(self):
        """Clean up resources"""
        if hasattr(self, 'pipe') and self.pipe is not None:
            del self.pipe
        if hasattr(self, 'controlnet') and self.controlnet is not None:
            del self.controlnet
        self.memory_optimizer.cleanup_memory()
        print("🧹 Generator resources cleaned up")

print("✅ Enhanced FLUX ControlNet Generator class defined")

In [ ]:
# @title 4. Initialize Enhanced FLUX Generator
# Initialize the FLUX ControlNet generator with configuration

try:
    print("🚀 Initializing Enhanced FLUX ControlNet Generator...")
    generator = FLUXControlNetGenerator(config, device='cuda')
    print("✅ FLUX Generator initialized successfully")
    
    # Display model information
    print("\n📋 Model Configuration:")
    print(f"   Base Model: {generator.base_model_id}")
    print(f"   ControlNet: {generator.controlnet_id}")
    print(f"   Mixed Precision: {generator.model_config.get('mixed_precision', True)}")
    print(f"   Device: {generator.device}")
    
except Exception as e:
    print(f"❌ Failed to initialize FLUX generator: {e}")
    print("\n🔧 Troubleshooting suggestions:")
    print("   1. Ensure you have sufficient GPU memory (16GB+ recommended)")
    print("   2. Try enabling CPU offloading in config.json")
    print("   3. Reduce batch size or image resolution")
    print("   4. Check internet connection for model downloads")
    raise

In [ ]:
# @title 5. Load Data and Setup Dynamic Batch Processing
import glob
from concurrent.futures import ThreadPoolExecutor
import threading

class DynamicBatchProcessor:
    """Dynamic batch processor with memory-aware sizing"""
    
    def __init__(self, generator: FLUXControlNetGenerator, config: Dict[str, Any]):
        self.generator = generator
        self.config = config
        self.memory_optimizer = MemoryOptimizer()
        self.data_validator = DataValidator()
        
        # Processing statistics
        self.stats = {
            'total_processed': 0,
            'successful': 0,
            'failed': 0,
            'total_time': 0.0,
            'quality_scores': []
        }
    
    def calculate_optimal_batch_size(self) -> int:
        """Calculate optimal batch size based on available memory"""
        if torch.cuda.is_available():
            # Get available GPU memory
            total_memory = torch.cuda.get_device_properties(0).total_memory
            allocated_memory = torch.cuda.memory_allocated()
            available_memory = (total_memory - allocated_memory) / (1024**3)  # GB
            
            # Estimate memory per image (FLUX.1-dev is memory intensive)
            memory_per_image = 2.5  # GB per 1024x1024 image
            
            # Calculate batch size with safety margin
            batch_size = max(1, int(available_memory * 0.7 / memory_per_image))
            
            print(f"📊 Memory Analysis:")
            print(f"   Available GPU Memory: {available_memory:.1f} GB")
            print(f"   Estimated per image: {memory_per_image} GB")
            print(f"   Optimal batch size: {batch_size}")
            
            return batch_size
        else:
            return 1
    
    def load_depth_maps_and_params(self) -> Tuple[List[np.ndarray], List[Dict[str, Any]]]:
        """Load depth maps and generation parameters"""
        print("📂 Loading depth maps and parameters...")
        
        # Try to load from previous pipeline stages
        try:
            # Load depth extraction results
            depth_results = DataFlowManager.load_previous_results(
                "depth_anything_v2_extraction",
                ["depth_maps.json", "depth_quality.json"]
            )
            
            # Load augmentation parameters
            augmentation_results = DataFlowManager.load_previous_results(
                "data_augmentation_system",
                ["augmentation_params.json"]
            )
            
            print(f"✅ Loaded {len(depth_results.get('depth_maps', []))} depth maps")
            print(f"✅ Loaded {len(augmentation_results.get('generation_parameters', []))} parameter sets")
            
            return depth_results.get('depth_maps', []), augmentation_results.get('generation_parameters', [])
            
        except Exception as e:
            print(f"⚠️ Could not load from previous stages: {e}")
            print("   Using fallback data loading...")
            
            # Fallback: look for depth map files
            depth_files = glob.glob("./data/depth_maps/*.npy")
            if not depth_files:
                depth_files = glob.glob("./outputs/depth_maps/*.npy")
            
            depth_maps = []
            for depth_file in depth_files[:10]:  # Limit for demo
                try:
                    depth_map = np.load(depth_file)
                    depth_maps.append(depth_map)
                except Exception as e:
                    print(f"⚠️ Could not load {depth_file}: {e}")
            
            # Create sample parameters if none available
            sample_params = [
                {
                    'prompt': 'A beautiful ocean beach scene with waves, photorealistic, high quality',
                    'height': 1024,
                    'width': 1024,
                    'num_inference_steps': 28,
                    'guidance_scale': 3.5,
                    'controlnet_conditioning_scale': 0.6,
                    'seed': 42 + i
                }
                for i in range(len(depth_maps))
            ]
            
            print(f"📋 Using {len(depth_maps)} depth maps with sample parameters")
            return depth_maps, sample_params
    
    def process_batch(self, depth_maps: List[np.ndarray], params_list: List[Dict[str, Any]], 
                     batch_size: int = None) -> List[EnhancedGenerationResult]:
        """Process a batch of images with dynamic sizing"""
        if batch_size is None:
            batch_size = self.calculate_optimal_batch_size()
        
        results = []
        total_items = min(len(depth_maps), len(params_list))
        
        print(f"🚀 Starting batch processing: {total_items} items, batch size: {batch_size}")
        
        # Create progress tracker
        progress = ProgressTracker(total_items, "Generating synthetic images")
        
        try:
            for i in range(0, total_items, batch_size):
                batch_end = min(i + batch_size, total_items)
                batch_depth_maps = depth_maps[i:batch_end]
                batch_params = params_list[i:batch_end]
                
                print(f"\n📦 Processing batch {i//batch_size + 1}: items {i+1}-{batch_end}")
                
                # Process each item in the batch
                for j, (depth_map, params) in enumerate(zip(batch_depth_maps, batch_params)):
                    try:
                        # Validate depth map quality
                        if depth_map.size == 0 or np.all(depth_map == 0):
                            print(f"⚠️ Skipping invalid depth map {i+j+1}")
                            self.stats['failed'] += 1
                            continue
                        
                        # Generate synthetic image
                        result = self.generator.generate_with_memory_management(depth_map, params)
                        
                        # Save result
                        image_filename = f"synthetic_{i+j+1:04d}.png"
                        image_path = IMAGES_DIR / image_filename
                        result.synthetic_image.save(image_path, "PNG")
                        
                        # Update statistics
                        self.stats['successful'] += 1
                        self.stats['total_time'] += result.processing_time
                        self.stats['quality_scores'].append(result.quality_metrics['overall_quality'])
                        
                        results.append(result)
                        
                        print(f"✅ Generated {image_filename} (Quality: {result.quality_metrics['overall_quality']:.3f})")
                        
                    except Exception as e:
                        print(f"❌ Failed to generate image {i+j+1}: {e}")
                        self.stats['failed'] += 1
                    
                    finally:
                        progress.update(1)
                        self.stats['total_processed'] += 1
                
                # Memory cleanup between batches
                self.memory_optimizer.cleanup_memory()
                
                # Display batch statistics
                if torch.cuda.is_available():
                    memory_used = torch.cuda.memory_allocated() / (1024**3)
                    print(f"📊 Batch complete. GPU Memory: {memory_used:.2f} GB")
        
        finally:
            progress.close()
        
        return results
    
    def get_processing_summary(self) -> Dict[str, Any]:
        """Get comprehensive processing summary"""
        avg_quality = np.mean(self.stats['quality_scores']) if self.stats['quality_scores'] else 0.0
        avg_time = self.stats['total_time'] / max(1, self.stats['successful'])
        
        return {
            'total_processed': self.stats['total_processed'],
            'successful': self.stats['successful'],
            'failed': self.stats['failed'],
            'success_rate': self.stats['successful'] / max(1, self.stats['total_processed']),
            'average_quality': avg_quality,
            'average_processing_time': avg_time,
            'total_processing_time': self.stats['total_time'],
            'quality_distribution': {
                'min': min(self.stats['quality_scores']) if self.stats['quality_scores'] else 0,
                'max': max(self.stats['quality_scores']) if self.stats['quality_scores'] else 0,
                'std': np.std(self.stats['quality_scores']) if self.stats['quality_scores'] else 0
            }
        }

# Initialize the batch processor
print("🔧 Initializing Dynamic Batch Processor...")
batch_processor = DynamicBatchProcessor(generator, config)

# Load data
depth_maps, generation_params = batch_processor.load_depth_maps_and_params()

if not depth_maps or not generation_params:
    print("❌ No data available for processing")
    print("   Please ensure previous pipeline stages have been completed")
else:
    print(f"✅ Ready to process {len(depth_maps)} depth maps with {len(generation_params)} parameter sets")

In [ ]:
# @title 6. Execute Enhanced Batch Processing
# Run the enhanced FLUX generation with dynamic batch sizing

if depth_maps and generation_params:
    print("🚀 Starting Enhanced FLUX Synthetic Generation...")
    print(f"📊 Processing {len(depth_maps)} depth maps")
    
    # Limit processing for demonstration (remove limit for full processing)
    max_images = min(len(depth_maps), config.get('processing', {}).get('max_images', 50))
    
    try:
        # Process the batch
        results = batch_processor.process_batch(
            depth_maps[:max_images], 
            generation_params[:max_images]
        )
        
        print(f"\n✅ Batch processing completed!")
        print(f"📈 Generated {len(results)} synthetic images")
        
        # Get processing summary
        summary = batch_processor.get_processing_summary()
        
        print("\n📊 Processing Summary:")
        print(f"   Total Processed: {summary['total_processed']}")
        print(f"   Successful: {summary['successful']}")
        print(f"   Failed: {summary['failed']}")
        print(f"   Success Rate: {summary['success_rate']:.1%}")
        print(f"   Average Quality: {summary['average_quality']:.3f}")
        print(f"   Average Time per Image: {summary['average_processing_time']:.1f}s")
        print(f"   Total Processing Time: {summary['total_processing_time']:.1f}s")
        
        # Save processing results
        results_metadata = {
            'stage_name': 'flux_controlnet_generation',
            'timestamp': pd.Timestamp.now().isoformat(),
            'model_info': {
                'base_model': generator.base_model_id,
                'controlnet': generator.controlnet_id,
                'mixed_precision': generator.model_config.get('mixed_precision', True)
            },
            'processing_summary': summary,
            'generated_images': [f"synthetic_{i+1:04d}.png" for i in range(len(results))],
            'configuration': config
        }
        
        # Save metadata
        metadata_file = METADATA_DIR / 'flux_generation_results.json'
        with open(metadata_file, 'w') as f:
            json.dump(results_metadata, f, indent=2, default=str)
        
        print(f"\n💾 Results saved to {IMAGES_DIR}")
        print(f"📋 Metadata saved to {metadata_file}")
        
    except Exception as e:
        print(f"❌ Batch processing failed: {e}")
        import traceback
        traceback.print_exc()
        
        # Save partial results if any
        summary = batch_processor.get_processing_summary()
        if summary['successful'] > 0:
            print(f"\n💾 Saving {summary['successful']} partial results...")
            partial_metadata = {
                'stage_name': 'flux_controlnet_generation_partial',
                'timestamp': pd.Timestamp.now().isoformat(),
                'error': str(e),
                'processing_summary': summary
            }
            
            partial_file = METADATA_DIR / 'flux_generation_partial.json'
            with open(partial_file, 'w') as f:
                json.dump(partial_metadata, f, indent=2, default=str)
else:
    print("❌ No data available for processing")
    print("   Please run previous pipeline stages first")

In [ ]:
# @title 7. Quality Analysis and Data Distribution Comparison
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as scipy_stats
import pandas as pd

class QualityAnalyzer:
    """Comprehensive quality analysis for synthetic vs real data"""
    
    def __init__(self, synthetic_results: List[EnhancedGenerationResult]):
        self.synthetic_results = synthetic_results
        self.analysis_results = {}
    
    def analyze_synthetic_quality(self) -> Dict[str, Any]:
        """Analyze quality metrics of synthetic images"""
        if not self.synthetic_results:
            return {'error': 'No synthetic results available'}
        
        # Extract quality metrics
        quality_data = {
            'brightness': [r.quality_metrics['brightness'] for r in self.synthetic_results],
            'contrast': [r.quality_metrics['contrast'] for r in self.synthetic_results],
            'color_variance': [r.quality_metrics['color_variance'] for r in self.synthetic_results],
            'edge_correlation': [r.quality_metrics['edge_correlation'] for r in self.synthetic_results],
            'overall_quality': [r.quality_metrics['overall_quality'] for r in self.synthetic_results]
        }
        
        # Calculate statistics
        analysis = {}
        for metric, values in quality_data.items():
            analysis[metric] = {
                'mean': np.mean(values),
                'std': np.std(values),
                'min': np.min(values),
                'max': np.max(values),
                'median': np.median(values),
                'q25': np.percentile(values, 25),
                'q75': np.percentile(values, 75)
            }
        
        return analysis
    
    def compare_with_reference_data(self, reference_images_dir: str = None) -> Dict[str, Any]:
        """Compare synthetic data distribution with reference real data"""
        comparison_results = {
            'synthetic_stats': self.analyze_synthetic_quality(),
            'comparison_available': False
        }
        
        if reference_images_dir and Path(reference_images_dir).exists():
            try:
                # Load reference images for comparison
                reference_files = list(Path(reference_images_dir).glob('*.jpg')) + \
                                list(Path(reference_images_dir).glob('*.png'))
                
                if reference_files:
                    print(f"📊 Analyzing {len(reference_files)} reference images...")
                    
                    # Analyze reference images
                    ref_metrics = []
                    for img_path in reference_files[:20]:  # Limit for performance
                        try:
                            img = Image.open(img_path)
                            img_array = np.array(img)
                            
                            # Calculate same metrics as synthetic
                            brightness = np.mean(img_array) / 255.0
                            contrast = np.std(img_array) / 255.0
                            color_variance = np.var(img_array, axis=(0, 1)).mean() / 255.0
                            
                            ref_metrics.append({
                                'brightness': brightness,
                                'contrast': contrast,
                                'color_variance': color_variance
                            })
                        except Exception as e:
                            print(f"⚠️ Could not analyze {img_path}: {e}")
                    
                    if ref_metrics:
                        # Calculate reference statistics
                        ref_stats = {}
                        for metric in ['brightness', 'contrast', 'color_variance']:
                            values = [m[metric] for m in ref_metrics]
                            ref_stats[metric] = {
                                'mean': np.mean(values),
                                'std': np.std(values)
                            }
                        
                        # Perform statistical comparison
                        synthetic_stats = comparison_results['synthetic_stats']
                        statistical_tests = {}
                        
                        for metric in ['brightness', 'contrast', 'color_variance']:
                            if metric in synthetic_stats:
                                # Calculate distribution similarity
                                syn_mean = synthetic_stats[metric]['mean']
                                ref_mean = ref_stats[metric]['mean']
                                
                                # Simple similarity score (1.0 = identical, 0.0 = completely different)
                                similarity = 1.0 - min(1.0, abs(syn_mean - ref_mean) / max(syn_mean, ref_mean, 0.001))
                                
                                statistical_tests[metric] = {
                                    'synthetic_mean': syn_mean,
                                    'reference_mean': ref_mean,
                                    'similarity_score': similarity,
                                    'difference': abs(syn_mean - ref_mean)
                                }
                        
                        comparison_results.update({
                            'reference_stats': ref_stats,
                            'statistical_comparison': statistical_tests,
                            'comparison_available': True,
                            'reference_count': len(ref_metrics)
                        })
                        
                        print("✅ Reference data comparison completed")
            
            except Exception as e:
                print(f"⚠️ Could not perform reference comparison: {e}")
        
        return comparison_results
    
    def generate_quality_report(self, save_path: str = None) -> str:
        """Generate comprehensive quality report"""
        analysis = self.analyze_synthetic_quality()
        
        report = []
        report.append("# FLUX Synthetic Image Quality Report")
        report.append(f"Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append(f"Total Images Analyzed: {len(self.synthetic_results)}")
        report.append("")
        
        report.append("## Quality Metrics Summary")
        for metric, stats in analysis.items():
            if isinstance(stats, dict):
                report.append(f"### {metric.replace('_', ' ').title()}")
                report.append(f"- Mean: {stats['mean']:.3f}")
                report.append(f"- Std Dev: {stats['std']:.3f}")
                report.append(f"- Range: {stats['min']:.3f} - {stats['max']:.3f}")
                report.append(f"- Median: {stats['median']:.3f}")
                report.append("")
        
        # Performance metrics
        if self.synthetic_results:
            avg_time = np.mean([r.processing_time for r in self.synthetic_results])
            total_time = sum([r.processing_time for r in self.synthetic_results])
            
            report.append("## Performance Metrics")
            report.append(f"- Average Generation Time: {avg_time:.1f} seconds")
            report.append(f"- Total Processing Time: {total_time:.1f} seconds")
            report.append(f"- Images per Minute: {60 / avg_time:.1f}")
            report.append("")
        
        # Memory usage
        if self.synthetic_results and 'memory_usage' in self.synthetic_results[0].__dict__:
            peak_memory = max([r.memory_usage.get('peak_gpu_gb', 0) for r in self.synthetic_results])
            report.append("## Memory Usage")
            report.append(f"- Peak GPU Memory: {peak_memory:.2f} GB")
            report.append("")
        
        report_text = "\n".join(report)
        
        if save_path:
            with open(save_path, 'w') as f:
                f.write(report_text)
            print(f"📄 Quality report saved to {save_path}")
        
        return report_text

# Perform quality analysis if we have results
if 'results' in locals() and results:
    print("📊 Performing Quality Analysis...")
    
    analyzer = QualityAnalyzer(results)
    
    # Analyze synthetic quality
    quality_analysis = analyzer.analyze_synthetic_quality()
    
    print("\n📈 Synthetic Image Quality Analysis:")
    for metric, stats in quality_analysis.items():
        if isinstance(stats, dict):
            print(f"   {metric.replace('_', ' ').title()}:")
            print(f"     Mean: {stats['mean']:.3f} ± {stats['std']:.3f}")
            print(f"     Range: [{stats['min']:.3f}, {stats['max']:.3f}]")
    
    # Try to compare with reference data
    reference_dirs = ['./data/real', './data/processed', './outputs/real']
    comparison_done = False
    
    for ref_dir in reference_dirs:
        if Path(ref_dir).exists():
            print(f"\n🔍 Comparing with reference data in {ref_dir}...")
            comparison = analyzer.compare_with_reference_data(ref_dir)
            
            if comparison['comparison_available']:
                print("\n📊 Synthetic vs Reference Comparison:")
                for metric, comp in comparison['statistical_comparison'].items():
                    print(f"   {metric.replace('_', ' ').title()}:")
                    print(f"     Synthetic: {comp['synthetic_mean']:.3f}")
                    print(f"     Reference: {comp['reference_mean']:.3f}")
                    print(f"     Similarity: {comp['similarity_score']:.1%}")
                comparison_done = True
                break
    
    if not comparison_done:
        print("\n⚠️ No reference data found for comparison")
        print("   Place real images in ./data/real/ for distribution comparison")
    
    # Generate and save quality report
    report_path = METADATA_DIR / 'quality_report.md'
    quality_report = analyzer.generate_quality_report(str(report_path))
    
    print(f"\n📄 Comprehensive quality report generated: {report_path}")
    
else:
    print("⚠️ No results available for quality analysis")

In [ ]:
# @title 8. Cleanup and Final Summary
import gc

# Cleanup resources
print("🧹 Cleaning up resources...")

try:
    if 'generator' in locals():
        generator.cleanup()
    
    # Clear large variables
    if 'results' in locals():
        del results
    if 'depth_maps' in locals():
        del depth_maps
    if 'generation_params' in locals():
        del generation_params
    
    # Force garbage collection
    gc.collect()
    
    # Clear GPU memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        final_memory = torch.cuda.memory_allocated() / (1024**3)
        print(f"📊 Final GPU Memory Usage: {final_memory:.2f} GB")
    
    print("✅ Cleanup completed")

except Exception as e:
    print(f"⚠️ Cleanup warning: {e}")

# Final summary
print("\n" + "="*60)
print("🎉 FLUX ControlNet Synthetic Generation Complete!")
print("="*60)

if 'batch_processor' in locals():
    final_summary = batch_processor.get_processing_summary()
    
    print(f"\n📊 Final Statistics:")
    print(f"   ✅ Successfully Generated: {final_summary['successful']} images")
    print(f"   ❌ Failed: {final_summary['failed']} images")
    print(f"   📈 Success Rate: {final_summary['success_rate']:.1%}")
    print(f"   ⭐ Average Quality Score: {final_summary['average_quality']:.3f}")
    print(f"   ⏱️ Total Processing Time: {final_summary['total_processing_time']:.1f} seconds")

print(f"\n📂 Output Location: {IMAGES_DIR}")
print(f"📋 Metadata Location: {METADATA_DIR}")

print("\n🚀 Next Steps:")
print("   1. Review generated images in the output directory")
print("   2. Check quality report for detailed analysis")
print("   3. Proceed to notebook 06 for model training")
print("   4. Use generated synthetic data for training augmentation")

print("\n" + "="*60)

In [ ]:
# @title 9. Enhanced Processing with Advanced Memory Management
# Import enhanced processing capabilities
import sys
sys.path.append('./utils')

try:
    from enhanced_batch_processor import EnhancedBatchProcessor
    from flux_memory_manager import FLUXMemoryManager, QualityValidator, DataDistributionComparator
    print("✅ Enhanced processing modules imported successfully")
except ImportError as e:
    print(f"⚠️ Could not import enhanced modules: {e}")
    print("   Falling back to basic processing...")
    # Use the basic batch processor defined earlier
    EnhancedBatchProcessor = None

# Initialize enhanced processor if available
if EnhancedBatchProcessor and 'generator' in locals():
    print("🚀 Initializing Enhanced Batch Processor...")
    
    enhanced_processor = EnhancedBatchProcessor(generator, config)
    
    # Load reference images for distribution comparison
    reference_images = []
    reference_dirs = ['./data/real', './data/processed', './outputs/real']
    
    for ref_dir in reference_dirs:
        ref_path = Path(ref_dir)
        if ref_path.exists():
            print(f"📂 Loading reference images from {ref_dir}...")
            
            # Load reference images
            for img_file in list(ref_path.glob('*.jpg'))[:10] + list(ref_path.glob('*.png'))[:10]:
                try:
                    img = Image.open(img_file)
                    # Resize to standard size for comparison
                    img = img.resize((512, 512), Image.Resampling.LANCZOS)
                    reference_images.append(img)
                except Exception as e:
                    print(f"⚠️ Could not load {img_file}: {e}")
            
            if reference_images:
                print(f"✅ Loaded {len(reference_images)} reference images")
                break
    
    if not reference_images:
        print("⚠️ No reference images found - distribution comparison will be skipped")
        print("   Place real images in ./data/real/ for comparison")
    
    # Run enhanced processing if data is available
    if 'depth_maps' in locals() and 'generation_params' in locals() and depth_maps:
        print("\n🚀 Starting Enhanced FLUX Processing with Quality Validation...")
        
        # Limit processing for demonstration
        max_images = min(len(depth_maps), config.get('processing', {}).get('max_images', 20))
        
        try:
            # Run enhanced batch processing
            enhanced_result = enhanced_processor.process_batch_with_quality_validation(
                depth_maps[:max_images],
                generation_params[:max_images],
                IMAGES_DIR,
                reference_images if reference_images else None
            )
            
            print("\n✅ Enhanced Processing Completed!")
            print(f"📊 Results Summary:")
            print(f"   Total Processed: {enhanced_result.total_processed}")
            print(f"   Successful: {enhanced_result.successful}")
            print(f"   Failed: {enhanced_result.failed}")
            print(f"   Success Rate: {enhanced_result.success_rate:.1%}")
            print(f"   Average Quality: {enhanced_result.quality_summary['mean_quality']:.3f}")
            print(f"   Peak GPU Memory: {enhanced_result.memory_usage['peak_gpu_usage']:.2f} GB")
            
            if enhanced_result.distribution_comparison:
                comp = enhanced_result.distribution_comparison
                print(f"   Distribution Similarity: {comp['overall_similarity']:.1%}")
                print(f"   Assessment: {comp['assessment']['level'].title()}")
            
            # Generate comprehensive report
            report = enhanced_processor.generate_processing_report(enhanced_result, IMAGES_DIR)
            print(f"\n📄 Comprehensive report generated: {IMAGES_DIR}/processing_report.md")
            
            # Display quality distribution
            if enhanced_result.quality_summary['mean_quality'] > 0:
                quality_assessment = "Excellent" if enhanced_result.quality_summary['mean_quality'] >= 0.8 else \
                                   "Good" if enhanced_result.quality_summary['mean_quality'] >= 0.6 else \
                                   "Acceptable" if enhanced_result.quality_summary['mean_quality'] >= 0.4 else "Poor"
                
                print(f"\n⭐ Overall Quality Assessment: {quality_assessment}")
                print(f"   Quality Range: [{enhanced_result.quality_summary['min_quality']:.3f}, {enhanced_result.quality_summary['max_quality']:.3f}]")
                print(f"   Acceptable Images: {enhanced_result.quality_summary['acceptable_count']}/{enhanced_result.successful}")
        
        except Exception as e:
            print(f"❌ Enhanced processing failed: {e}")
            import traceback
            traceback.print_exc()
            
            # Fall back to basic processing
            print("\n🔄 Falling back to basic processing...")
            if 'batch_processor' in locals():
                basic_results = batch_processor.process_batch(
                    depth_maps[:max_images], 
                    generation_params[:max_images]
                )
                print(f"✅ Basic processing completed: {len(basic_results)} images generated")
    
    else:
        print("❌ No data available for enhanced processing")
        print("   Please ensure previous pipeline stages have been completed")

else:
    print("⚠️ Enhanced processing not available - using basic processing")
    # Use basic processing if enhanced is not available
    if 'batch_processor' in locals() and 'depth_maps' in locals():
        max_images = min(len(depth_maps), 10)
        basic_results = batch_processor.process_batch(
            depth_maps[:max_images], 
            generation_params[:max_images]
        )
        print(f"✅ Basic processing completed: {len(basic_results)} images generated")

In [ ]:
# @title 10. Advanced Quality Validation and Analysis
# Perform detailed quality analysis on generated images

if 'enhanced_result' in locals() and enhanced_result.successful > 0:
    print("📊 Performing Advanced Quality Analysis...")
    
    # Load generated images for detailed analysis
    generated_images = []
    quality_scores = []
    
    for filename in enhanced_result.generated_files[:10]:  # Analyze first 10 for performance
        try:
            img_path = IMAGES_DIR / filename
            if img_path.exists():
                img = Image.open(img_path)
                generated_images.append(img)
                
                # Load corresponding metadata
                metadata_file = IMAGES_DIR / "metadata" / f"{filename.replace('.png', '_metadata.json')}"
                if metadata_file.exists():
                    with open(metadata_file, 'r') as f:
                        metadata = json.load(f)
                        quality_scores.append(metadata.get('quality_metrics', {}).get('overall_score', 0.0))
        except Exception as e:
            print(f"⚠️ Could not analyze {filename}: {e}")
    
    if generated_images:
        print(f"\n📈 Detailed Analysis of {len(generated_images)} Generated Images:")
        
        # Quality distribution analysis
        if quality_scores:
            quality_array = np.array(quality_scores)
            
            print(f"\n🎯 Quality Score Distribution:")
            print(f"   Mean: {np.mean(quality_array):.3f} ± {np.std(quality_array):.3f}")
            print(f"   Median: {np.median(quality_array):.3f}")
            print(f"   Range: [{np.min(quality_array):.3f}, {np.max(quality_array):.3f}]")
            
            # Quality categories
            excellent = np.sum(quality_array >= 0.8)
            good = np.sum((quality_array >= 0.6) & (quality_array < 0.8))
            acceptable = np.sum((quality_array >= 0.4) & (quality_array < 0.6))
            poor = np.sum(quality_array < 0.4)
            
            print(f"\n📊 Quality Categories:")
            print(f"   Excellent (≥0.8): {excellent} ({excellent/len(quality_scores)*100:.1f}%)")
            print(f"   Good (0.6-0.8): {good} ({good/len(quality_scores)*100:.1f}%)")
            print(f"   Acceptable (0.4-0.6): {acceptable} ({acceptable/len(quality_scores)*100:.1f}%)")
            print(f"   Poor (<0.4): {poor} ({poor/len(quality_scores)*100:.1f}%)")
        
        # Memory efficiency analysis
        if 'enhanced_result' in locals():
            memory_usage = enhanced_result.memory_usage
            processing_time = enhanced_result.processing_time
            
            print(f"\n⚡ Performance Metrics:")
            print(f"   Images per Second: {enhanced_result.successful / processing_time:.2f}")
            print(f"   Average Memory per Image: {memory_usage['average_gpu_usage']:.2f} GB")
            print(f"   Memory Efficiency: {enhanced_result.successful / memory_usage['peak_gpu_usage']:.1f} images/GB")
        
        # Distribution comparison summary
        if enhanced_result.distribution_comparison:
            comp = enhanced_result.distribution_comparison
            
            print(f"\n🔍 Synthetic vs Real Data Comparison:")
            print(f"   Overall Similarity: {comp['overall_similarity']:.1%}")
            print(f"   Assessment: {comp['assessment']['message']}")
            
            # Feature-specific similarities
            print(f"\n📋 Feature Similarities:")
            for feature, comparison in comp['feature_comparisons'].items():
                similarity = comparison['similarity_score']
                status = "✅" if similarity >= 0.7 else "⚠️" if similarity >= 0.5 else "❌"
                print(f"   {status} {feature.replace('_', ' ').title()}: {similarity:.1%}")
        
        # Recommendations based on analysis
        print(f"\n💡 Recommendations:")
        
        if quality_scores:
            avg_quality = np.mean(quality_array)
            if avg_quality >= 0.8:
                print(f"   ✅ Excellent quality achieved - current settings are optimal")
            elif avg_quality >= 0.6:
                print(f"   📈 Good quality - consider fine-tuning generation parameters")
            elif avg_quality >= 0.4:
                print(f"   ⚠️ Acceptable quality - review depth map quality and prompts")
            else:
                print(f"   ❌ Poor quality - check input data and model configuration")
        
        if enhanced_result.distribution_comparison:
            similarity = enhanced_result.distribution_comparison['overall_similarity']
            if similarity >= 0.8:
                print(f"   ✅ Excellent distribution match - synthetic data is highly realistic")
            elif similarity >= 0.6:
                print(f"   📊 Good distribution match - minor adjustments may improve realism")
            else:
                print(f"   ⚠️ Distribution mismatch - review generation parameters and prompts")
        
        # Memory optimization recommendations
        if memory_usage['peak_gpu_usage'] > 12:
            print(f"   🧠 High memory usage detected - consider enabling CPU offloading")
        elif memory_usage['peak_gpu_usage'] < 6:
            print(f"   🚀 Low memory usage - can increase batch size for better efficiency")
    
    else:
        print("⚠️ No generated images found for analysis")

else:
    print("⚠️ No enhanced results available for analysis")
    print("   Run the enhanced processing cell first")